In [1]:
from simple.ohlc import debounce, midPrice, TTrade, npJoin
from simple.chart import chartFigure
from simple.funcs import symlog
import numpy as np

In [2]:
T = np.load('data/ETHUSDT.2022-05-08.tick.npz')['ETHUSDT'].view(np.recarray)
T[-5:]

rec.array([('2022-05-08T05:59:59.886999', 2537.16, 394.),
           ('2022-05-08T05:59:59.886999', 2537.16, 394.),
           ('2022-05-08T05:59:59.886999', 2537.16, 210.),
           ('2022-05-08T05:59:59.897000', 2537.16,   2.),
           ('2022-05-08T05:59:59.909999', 2537.16,   2.)],
          dtype=[('DateTime', '<M8[us]'), ('Price', '<f8'), ('Size', '<f8')])

In [3]:
Buy = T[T.Size > 0]
Sell = T[T.Size < 0]

In [4]:
C = debounce(T)[:-1]
C[-5:]

rec.array([('2022-05-08T05:59:50.540000', 155085, 2537.085,       0,  1.3485e+04, 13485.,  0.000e+00,  4,  4,  0),
           ('2022-05-08T05:59:50.540000', 155086, 2537.145,       0,  2.5350e+03,  2535.,  0.000e+00,  3,  3,  0),
           ('2022-05-08T05:59:50.611000', 155087, 2537.185,   71000,  2.3760e+03,  2378., -2.000e+00,  6,  5,  1),
           ('2022-05-08T05:59:51.163000', 155088, 2537.105,   18000, -1.0000e+00,   197., -1.980e+02,  2,  1,  1),
           ('2022-05-08T05:59:59.780999', 155089, 2536.985, 8617999,  3.3760e+03,  9984., -6.608e+03, 67, 55, 12)],
          dtype=[('DateTime', '<M8[us]'), ('Index', '<i8'), ('Price', '<f8'), ('Duration', '<m8[us]'), ('Size', '<f8'), ('BuySize', '<f8'), ('SellSize', '<f8'), ('Count', '<i8'), ('BuyCount', '<i8'), ('SellCount', '<i8')])

In [5]:
len(T), len(C)

(907307, 155090)

In [6]:
# datetime chart
fig = chartFigure(height=400,
    Sell=dict(x=Sell.DateTime, y=Sell.Price, color='red', mode='markers', size=-symlog(Sell.Size)+3, opacity=0.2),
    Buy=dict(x=Buy.DateTime, y=Buy.Price, color='green', mode='markers', size=symlog(Buy.Size)+3, opacity=0.2),
    Debounce=dict(mode='markers+lines', x=C.DateTime, y=C.Price, color='black', connectgaps=True, opacity=0.5)
)
fig

FigureWidgetResampler({
    'data': [{'marker': {'color': 'red',
                         'opacity': 0.2,
                         'size': array([ 7.58351894,  8.91265489,  4.69314718, ...,  9.22035583, 10.3403593 ,
                                         4.69314718])},
              'mode': 'markers',
              'name': '<b style="color:sandybrown">[R]</b> Sell <i style="color:#fc9944">~22s</i>',
              'type': 'scattergl',
              'uid': 'c5cf04a2-58b9-49d5-ba71-e576c58628ef',
              'x': array([datetime.datetime(2022, 5, 8, 0, 1, 0, 865000),
                          datetime.datetime(2022, 5, 8, 0, 1, 31, 602999),
                          datetime.datetime(2022, 5, 8, 0, 1, 36, 945999), ...,
                          datetime.datetime(2022, 5, 8, 5, 58, 41, 166000),
                          datetime.datetime(2022, 5, 8, 5, 59, 8, 347000),
                          datetime.datetime(2022, 5, 8, 5, 59, 56, 864999)], dtype=object),
              'y': array([2

In [7]:
C.Volume = C.BuySize + C.SellSize
Buy = C[C.Volume > 0]
Sell = C[C.Volume < 0]
Sell[-5:]

rec.array([('2022-05-08T05:59:42.346000', 155074, 2536.975,      0, -1.000e+02,   0.,  -100., 1, 0, 1),
           ('2022-05-08T05:59:42.346000', 155075, 2536.935,      0, -1.970e+03,   0., -1970., 3, 0, 3),
           ('2022-05-08T05:59:42.346000', 155076, 2536.905,      0, -5.081e+03,   0., -5081., 6, 0, 6),
           ('2022-05-08T05:59:42.745000', 155077, 2536.875, 363000, -5.649e+03,   0., -5649., 7, 0, 7),
           ('2022-05-08T05:59:51.163000', 155088, 2537.105,  18000, -1.000e+00, 197.,  -198., 2, 1, 1)],
          dtype=[('DateTime', '<M8[us]'), ('Index', '<i8'), ('Price', '<f8'), ('Duration', '<m8[us]'), ('Size', '<f8'), ('BuySize', '<f8'), ('SellSize', '<f8'), ('Count', '<i8'), ('BuyCount', '<i8'), ('SellCount', '<i8')])

In [8]:
S = np.load('data/ETHUSDT.2022-05-08.bidask.npz')['ETHUSDT'].view(np.recarray)
S[:5]

rec.array([('2022-05-08T00:01:00.007000', 2636., 2635.99),
           ('2022-05-08T00:01:00.043000', 2636., 2635.99),
           ('2022-05-08T00:01:00.079000', 2636., 2635.99),
           ('2022-05-08T00:01:00.117000', 2636., 2635.99),
           ('2022-05-08T00:01:00.149000', 2636., 2635.99)],
          dtype={'names': ['DateTime', 'Ask', 'Bid'], 'formats': ['<M8[us]', '<f4', '<f4'], 'offsets': [0, 8, 16], 'itemsize': 24})

In [9]:
K = npJoin(C.DateTime, S.DateTime)

In [10]:
# indexed chart with bidask
chartFigure(
    Sell=dict(x=Sell.Index, y=Sell.Price, color='red', mode='markers', size=symlog(-Sell.Size)-1, opacity=0.75),
    Buy=dict(x=Buy.Index, y=Buy.Price, color='green', mode='markers', size=symlog(Buy.Size)-1, opacity=0.75),
    Bid=dict(x=C.Index, y=S.Bid[K], color='green', opacity=0.4),
    Ask=dict(x=C.Index, y=S.Ask[K], color='red', opacity=0.4)
)

FigureWidgetResampler({
    'data': [{'marker': {'color': 'red',
                         'opacity': 0.75,
                         'size': array([1.60943791, 5.55682806, 8.08023742, ..., 8.53326337, 8.63923382,
                                        0.        ])},
              'mode': 'markers',
              'name': '<b style="color:sandybrown">[R]</b> Sell <i style="color:#fc9944">~155</i>',
              'type': 'scattergl',
              'uid': '0f68f23f-3cd4-4e2c-bbe4-b65653015903',
              'x': array([     6,    135,    261, ..., 154835, 154915, 155088]),
              'y': array([2636.155, 2639.285, 2639.565, ..., 2534.185, 2537.485, 2537.105])},
             {'marker': {'color': 'green',
                         'opacity': 0.75,
                         'size': array([8.18172046, 1.60943791, 4.99721227, ..., 7.83794892, 7.77317368,
                                        8.12444686])},
              'mode': 'markers',
              'name': '<b style="color:sandybrown">

In [12]:
# fig.add_bar(name='Buy', x=Buy.Index, y=Buy.Size, row=2, col=1, marker_color='green')
# fig.add_bar(name='Sell', x=Sell.Index, y=-Sell.Size, row=2, col=1, marker_color='red')